In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License"); you may
# not use this file except in compliance with the License. You may obtain
# a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS, WITHOUT
# WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the
# License for the specific language governing permissions and limitations
# under the License.

**Init**
- https://docs.google.com/document/d/1JRy6SiN6zs4Ib-n28Qs4jjEmblWQ6S3cc3tpWaj2_pE/edit?usp=sharing

**Reference** : 
- https://medium.com/ymedialabs-innovation/apache-spark-on-a-multi-node-cluster-b75967c8cb2b=0
- https://blog.sicara.com/get-started-pyspark-jupyter-guide-tutorial-ae2fe84f594f
- https://github.com/IBM/elasticsearch-spark-recommender

**Python** :
- version : 3.6

**Spark** :
- version : 2.2

**Elasticsearch**
- version : 5.3

**Elasticsearch-hadoop**
- version : 5.3

# Creating a Scalable Recommender with Apache Spark & Elasticsearch

In this notebook, you will create a recommendation engine using Spark and Elasticsearch. Using some movie rating data,
you will train a collaborative filtering model in Spark and export the trained model to Elasticsearch. Once exported, 
you can test your recommendations by querying Elasticsearch and displaying the results.

### _Prerequisites_

The notebook assumes you have installed Elasticsearch, the Elasticsearch vector-scoring plugin, Apache Spark and the Elasticsearch Spark connector detailed in the [setup steps](https://github.com/MLnick/elasticsearch-spark-recommender-demo/tree/master#steps).

> _Optional:_

> In order to display the images in the recommendation demo, you will need to access [The Movie Database (TMdb) API](https://www.themoviedb.org/documentation/api). Please follow the [instructions](https://developers.themoviedb.org/3/getting-started) to get an API key.

## Overview

You will work through the following steps

1. Pyspark init
2. Prepare the data
3. Transform Pyspark df to Pandas df
4. Content based Recommendation
5. Convert Pandas df to Pyspark df
6. Load Data into Elasticsearch
7. Train a recommmender model on the ratings data
8. Export ALS user and item factor vectors to Elasticsearch and train a collaborative filtering recommendation model using Spark MLlib
9. Save the model to Elasticsearch

## Step 1: Pyspark Init

In [1]:
from _spark_init import *
spark = create_spark_session('movie')
#spark.stop()
spark

## Step 2: Prepare the data

* This notebook uses the "small" version of the latest MovieLens movie rating dataset, containing about 100,000 ratings, 9,000 movies and 700 users
* The latest version of the data can be downloaded at https://grouplens.org/datasets/movielens/latest/
* Download the `ml-latest-small.zip` file and unzip it to a suitable location on your system.

The folder should contain a number of CSV files. We will be using the following files:
* `ratings.csv` - movie rating data
* `links.csv` - external database ids for each movie
* `movies.csv` - movie title and genres

In [2]:
# first import a few utility methods that we'll use later on
from IPython.display import Image, HTML, display

In [3]:
# if you unzipped the data to a different location than that specified in the Journey setup steps
# you can change the path below to point to the correct location
PATH_TO_DATA = "./data"

### Load rating and movie data

**Ratings**

The ratings data consists of around 100,000 ratings given by users to movies. Each row of the `DataFrame` consists of a `userId`, `movieId` and `timestamp` for the event, together with the `rating` given by the user to the movie

In [4]:
# load ratings data
ratings = spark.read.csv(PATH_TO_DATA + "/ratings.csv", header=True, inferSchema=True)
ratings.cache()
print("Number of ratings: %i" % ratings.count())
print("Sample of ratings:")
ratings.show(5)

Number of ratings: 100836
Sample of ratings:
+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [5]:
ratings = ratings.select(
    ratings.userId, ratings.movieId, ratings.rating, (ratings.timestamp.cast("long") * 1000).alias("timestamp"))
ratings.show(5)

+------+-------+------+------------+
|userId|movieId|rating|   timestamp|
+------+-------+------+------------+
|     1|      1|   4.0|964982703000|
|     1|      3|   4.0|964981247000|
|     1|      6|   4.0|964982224000|
|     1|     47|   5.0|964983815000|
|     1|     50|   5.0|964982931000|
+------+-------+------+------------+
only showing top 5 rows



**Movies**

The file `movies.csv` contains the `movieId`, `title` and `genres` for each movie. As you can see, the `genres` field is a bit tricky to use, as the genres are in the form of one string delimited by the `|` character: `Adventure|Animation|Children|Comedy|Fantasy`.

In [6]:
# load raw data from CSV
raw_movies = spark.read.csv(PATH_TO_DATA + "/movies.csv", header=True, inferSchema=True)
print("Raw movie data:")
raw_movies.select("movieId", "title", "genres").show(5, truncate=False)

Raw movie data:
+-------+----------------------------------+-------------------------------------------+
|movieId|title                             |genres                                     |
+-------+----------------------------------+-------------------------------------------+
|1      |Toy Story (1995)                  |Adventure|Animation|Children|Comedy|Fantasy|
|2      |Jumanji (1995)                    |Adventure|Children|Fantasy                 |
|3      |Grumpier Old Men (1995)           |Comedy|Romance                             |
|4      |Waiting to Exhale (1995)          |Comedy|Drama|Romance                       |
|5      |Father of the Bride Part II (1995)|Comedy                                     |
+-------+----------------------------------+-------------------------------------------+
only showing top 5 rows



Create a `DataFrame` user-defined function (UDF) to extract this delimited string into a list of genres.

In [7]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

In [8]:
extract_genres = udf(lambda x: x.lower().split("|"), ArrayType(StringType()))

In [9]:
import re
# define a UDF to extract the release year from the title, and return the new title and year in a struct type
def extract_year_fn(title):
    result = re.search("\(\d{4}\)", title)
    try:
        if result:
            group = result.group()
            year = group[1:-1]
            start_pos = result.start()
            title = title[:start_pos-1]
            return (title, year)
        else:
            return (title, 1970)
    except:
        print(title)

extract_year = udf(extract_year_fn, StructType([StructField("title", StringType(), True), StructField("release_date", StringType(), True)]))
    
# test out our function
s = "Jumanji (1995)"
extract_year_fn(s)

('Jumanji', '1995')

In [10]:
movies = raw_movies.select(
    "movieId", extract_year("title").title.alias("title"),\
    extract_year("title").release_date.alias("release_date"),\
    extract_genres("genres").alias("genres"))
print("Cleaned movie data:")
movies.show(5, truncate=False)

Cleaned movie data:
+-------+---------------------------+------------+-------------------------------------------------+
|movieId|title                      |release_date|genres                                           |
+-------+---------------------------+------------+-------------------------------------------------+
|1      |Toy Story                  |1995        |[adventure, animation, children, comedy, fantasy]|
|2      |Jumanji                    |1995        |[adventure, children, fantasy]                   |
|3      |Grumpier Old Men           |1995        |[comedy, romance]                                |
|4      |Waiting to Exhale          |1995        |[comedy, drama, romance]                         |
|5      |Father of the Bride Part II|1995        |[comedy]                                         |
+-------+---------------------------+------------+-------------------------------------------------+
only showing top 5 rows



Next, join the `links.csv` data to `movies` so that there is an id for _The Movie Database_ corresponding to each movie. You can use this id to retrieve movie poster images when displaying your recommendations later.

### Movie Data Extend
**links**

In [11]:
link_data = spark.read.csv(PATH_TO_DATA + "/links.csv", header=True, inferSchema=True)
link_data.show(5)

+-------+------+------+
|movieId|imdbId|tmdbId|
+-------+------+------+
|      1|114709|   862|
|      2|113497|  8844|
|      3|113228| 15602|
|      4|114885| 31357|
|      5|113041| 11862|
+-------+------+------+
only showing top 5 rows



In [12]:
# join movies with links to get TMDB id
movie_data = movies.join(link_data, movies.movieId == link_data.movieId, how='left')\
    .select(movies.movieId, movies.title, movies.release_date, movies.genres, link_data.tmdbId)
num_movies = movie_data.count()
print("Cleaned movie data with tmdbId links:")
movie_data.show(5, truncate=False)

Cleaned movie data with tmdbId links:
+-------+---------------------------+------------+-------------------------------------------------+------+
|movieId|title                      |release_date|genres                                           |tmdbId|
+-------+---------------------------+------------+-------------------------------------------------+------+
|1      |Toy Story                  |1995        |[adventure, animation, children, comedy, fantasy]|862   |
|2      |Jumanji                    |1995        |[adventure, children, fantasy]                   |8844  |
|3      |Grumpier Old Men           |1995        |[comedy, romance]                                |15602 |
|4      |Waiting to Exhale          |1995        |[comedy, drama, romance]                         |31357 |
|5      |Father of the Bride Part II|1995        |[comedy]                                         |11862 |
+-------+---------------------------+------------+------------------------------------------------

In [13]:
movie_data.dtypes

[('movieId', 'int'),
 ('title', 'string'),
 ('release_date', 'string'),
 ('genres', 'array<string>'),
 ('tmdbId', 'int')]

**Meta Data**

In [14]:
# load raw meta from CSV
raw_meta = spark.read.csv(PATH_TO_DATA + "/meta.csv", header=True, inferSchema=True)
print("Raw meta data:")
raw_meta.select(raw_meta.tmdbId, raw_meta.poster_path).show(5, truncate=False)

Raw meta data:
+------+--------------------------------+
|tmdbId|poster_path                     |
+------+--------------------------------+
|862   |/rhIRbceoE9lR4veEXuwCC2wARtG.jpg|
|8844  |/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg|
|15602 |/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg|
|31357 |/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg|
|11862 |/e64sOI48hQXyru7naBFyssKFxVd.jpg|
+------+--------------------------------+
only showing top 5 rows



In [15]:
raw_meta.dtypes

[('tmdbId', 'string'),
 ('poster_path', 'string'),
 ('runtime', 'int'),
 ('budget', 'string'),
 ('popularity', 'string'),
 ('revenue', 'bigint'),
 ('vote_average', 'double'),
 ('vote_count', 'int')]

In [16]:
meta_data = raw_meta.select(raw_meta.tmdbId.cast('int'), 
                            raw_meta.poster_path, 
                            raw_meta.runtime.cast('int'), 
                            raw_meta.budget.cast('int'), 
                            raw_meta.popularity.cast('int'), 
                            raw_meta.revenue.cast('int'), 
                            raw_meta.vote_average.cast('float'), 
                            raw_meta.vote_count.cast('int'))

In [17]:
meta_data.show(20)

+------+--------------------+-------+--------+----------+---------+------------+----------+
|tmdbId|         poster_path|runtime|  budget|popularity|  revenue|vote_average|vote_count|
+------+--------------------+-------+--------+----------+---------+------------+----------+
|   862|/rhIRbceoE9lR4veE...|     81|30000000|        21|373554033|         7.7|      5415|
|  8844|/vzmL6fP7aPKNKPRT...|    104|65000000|        17|262797249|         6.9|      2413|
| 15602|/6ksm1sjKMFLbO7UY...|    101|       0|        11|        0|         6.5|        92|
| 31357|/16XOMpEaLWkrcPqS...|    127|16000000|         3| 81452156|         6.1|        34|
| 11862|/e64sOI48hQXyru7n...|    106|       0|         8| 76578911|         5.7|       173|
|   949|/zMyfPUelumio3tiD...|    170|60000000|        17|187436818|         7.7|      1886|
| 11860|/jQh15y5YB7bWz1Nt...|    127|58000000|         6|        0|         6.2|       141|
| 45325|/sGO5Qa55p7wTu7FJ...|     97|       0|         2|        0|         5.4|

In [18]:
meta_data.dtypes

[('tmdbId', 'int'),
 ('poster_path', 'string'),
 ('runtime', 'int'),
 ('budget', 'int'),
 ('popularity', 'int'),
 ('revenue', 'int'),
 ('vote_average', 'float'),
 ('vote_count', 'int')]

### Movie Data + Meta Data

In [19]:
movie_cb = movie_data.join(meta_data, movie_data.tmdbId == meta_data.tmdbId, how='left')\
    .select(movie_data.movieId, movie_data.tmdbId, movie_data.title, movie_data.release_date, movie_data.genres, meta_data.poster_path, meta_data.runtime, meta_data.budget, meta_data.popularity, meta_data.revenue, meta_data.vote_average, meta_data.vote_count)

In [20]:
movie_cb.toPandas().head(5)

,movieId,tmdbId,title,release_date,genres,poster_path,runtime,budget,popularity,revenue,vote_average,vote_count
0,1,862.0,Toy Story,1995,"[adventure, animation, children, comedy, fantasy]",/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,81.0,30000000.0,21.0,373554033.0,7.7,5415.0
1,2,8844.0,Jumanji,1995,"[adventure, children, fantasy]",/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,104.0,65000000.0,17.0,262797249.0,6.9,2413.0
2,3,15602.0,Grumpier Old Men,1995,"[comedy, romance]",/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,101.0,0.0,11.0,0.0,6.5,92.0
3,4,31357.0,Waiting to Exhale,1995,"[comedy, drama, romance]",/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,127.0,16000000.0,3.0,81452156.0,6.1,34.0
4,5,11862.0,Father of the Bride Part II,1995,[comedy],/e64sOI48hQXyru7naBFyssKFxVd.jpg,106.0,0.0,8.0,76578911.0,5.7,173.0


In [21]:
movie_cb.dtypes

[('movieId', 'int'),
 ('tmdbId', 'int'),
 ('title', 'string'),
 ('release_date', 'string'),
 ('genres', 'array<string>'),
 ('poster_path', 'string'),
 ('runtime', 'int'),
 ('budget', 'int'),
 ('popularity', 'int'),
 ('revenue', 'int'),
 ('vote_average', 'float'),
 ('vote_count', 'int')]

##  Step 3: Spark Dataframe to Pandas Dataframe

In [22]:
df_cb = movie_cb.toPandas()

In [23]:
df_cb.head(2)

,movieId,tmdbId,title,release_date,genres,poster_path,runtime,budget,popularity,revenue,vote_average,vote_count
0,1,862.0,Toy Story,1995,"[adventure, animation, children, comedy, fantasy]",/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,81.0,30000000.0,21.0,373554033.0,7.7,5415.0
1,2,8844.0,Jumanji,1995,"[adventure, children, fantasy]",/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,104.0,65000000.0,17.0,262797249.0,6.9,2413.0


In [24]:
df_cb[df_cb['tmdbId'].isna()]

,movieId,tmdbId,title,release_date,genres,poster_path,runtime,budget,popularity,revenue,vote_average,vote_count
624,791,NaN,"Last Klezmer: Leopold Kozlowski, His Life and ...",1994,[documentary],None,NaN,NaN,NaN,NaN,NaN,NaN
844,1107,NaN,Loser,1991,[comedy],None,NaN,NaN,NaN,NaN,NaN,NaN
2143,2851,NaN,Saturn 3,1980,"[adventure, sci-fi, thriller]",None,NaN,NaN,NaN,NaN,NaN,NaN
3029,4051,NaN,Horrors of Spider Island (Ein Toter Hing im Netz),1960,"[horror, sci-fi]",None,NaN,NaN,NaN,NaN,NaN,NaN
5541,26587,NaN,"Decalogue, The (Dekalog)",1989,"[crime, drama, romance]",None,NaN,NaN,NaN,NaN,NaN,NaN
5863,32600,NaN,Eros,2004,[drama],None,NaN,NaN,NaN,NaN,NaN,NaN
6068,40697,NaN,Babylon 5,1970,[sci-fi],None,NaN,NaN,NaN,NaN,NaN,NaN
7392,79299,NaN,"No. 1 Ladies' Detective Agency, The",2008,"[comedy, crime, mystery]",None,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df_cb.shape

(9753, 12)

**Fillna**

In [26]:
df_cb.loc[df_cb['tmdbId'].isna(), 'tmdbId'] = 0

In [27]:
df_cb.loc[df_cb['poster_path'].isna(), 'poster_path'] = "NA"

In [28]:
df_cb.loc[df_cb['runtime'].isna(), 'runtime'] = 0

In [29]:
df_cb.loc[df_cb['budget'].isna(), 'budget'] = 0
df_cb.loc[df_cb['popularity'].isna(), 'popularity'] = 0
df_cb.loc[df_cb['revenue'].isna(), 'revenue'] = 0
df_cb.loc[df_cb['vote_average'].isna(), 'vote_average'] = 0
df_cb.loc[df_cb['vote_count'].isna(), 'vote_count'] = 0

In [30]:
df_cb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9753 entries, 0 to 9752
Data columns (total 12 columns):
movieId         9753 non-null int32
tmdbId          9753 non-null float64
title           9753 non-null object
release_date    9753 non-null object
genres          9753 non-null object
poster_path     9753 non-null object
runtime         9753 non-null float64
budget          9753 non-null float64
popularity      9753 non-null float64
revenue         9753 non-null float64
vote_average    9753 non-null float32
vote_count      9753 non-null float64
dtypes: float32(1), float64(6), int32(1), object(4)
memory usage: 838.2+ KB


**Transform dtype**

In [31]:
df_cb['tmdbId'] = df_cb['tmdbId'].astype(int)
df_cb['runtime'] = df_cb['runtime'].astype(int)
df_cb['budget'] = df_cb['budget'].astype(int)
df_cb['popularity'] = df_cb['popularity'].astype(int)
df_cb['revenue'] = df_cb['revenue'].astype(int)
df_cb['vote_count'] = df_cb['vote_count'].astype(int)

In [32]:
df_cb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9753 entries, 0 to 9752
Data columns (total 12 columns):
movieId         9753 non-null int32
tmdbId          9753 non-null int64
title           9753 non-null object
release_date    9753 non-null object
genres          9753 non-null object
poster_path     9753 non-null object
runtime         9753 non-null int64
budget          9753 non-null int64
popularity      9753 non-null int64
revenue         9753 non-null int64
vote_average    9753 non-null float32
vote_count      9753 non-null int64
dtypes: float32(1), int32(1), int64(6), object(4)
memory usage: 838.2+ KB


In [33]:
df_cb.head(5)

,movieId,tmdbId,title,release_date,genres,poster_path,runtime,budget,popularity,revenue,vote_average,vote_count
0,1,862,Toy Story,1995,"[adventure, animation, children, comedy, fantasy]",/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,81,30000000,21,373554033,7.7,5415
1,2,8844,Jumanji,1995,"[adventure, children, fantasy]",/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,104,65000000,17,262797249,6.9,2413
2,3,15602,Grumpier Old Men,1995,"[comedy, romance]",/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,101,0,11,0,6.5,92
3,4,31357,Waiting to Exhale,1995,"[comedy, drama, romance]",/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,127,16000000,3,81452156,6.1,34
4,5,11862,Father of the Bride Part II,1995,[comedy],/e64sOI48hQXyru7naBFyssKFxVd.jpg,106,0,8,76578911,5.7,173


In [34]:
df_cb.shape

(9753, 12)

In [35]:
df_cb['tmdbId'].dtype

dtype('int64')

## Step 4: Content Based Recommendation

In [36]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [37]:
md = pd.read_csv(PATH_TO_DATA + '/movies_metadata.csv')
md.head(2)

/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [38]:
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [39]:
md.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [40]:
#Check EDA Notebook for how and why I got these indices.

"""
those data are no id

"""

md = md.drop([19730, 29503, 35587])

**Metadata Based Recommender**

In [41]:
credits = pd.read_csv(PATH_TO_DATA + '/credits_small.csv')
keywords = pd.read_csv(PATH_TO_DATA + '/keywords_small.csv')

In [42]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

In [43]:
md.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [44]:
keywords.head(2)

,id,keywords
0,862,"['jealousy', 'toy', 'boy', 'friendship', 'frie..."
1,8844,"['board game', 'disappearance', ""based on chil..."


In [45]:
credits.head(2)

,id,director,cast
0,862,John Lasseter,"['Tom Hanks', 'Tim Allen', 'Don Rickles']"
1,8844,Joe Johnston,"['Robin Williams', 'Jonathan Hyde', 'Kirsten D..."


In [46]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

In [47]:
md.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,director,cast,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,John Lasseter,"['Tom Hanks', 'Tim Allen', 'Don Rickles']","['jealousy', 'toy', 'boy', 'friendship', 'frie..."
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,Joe Johnston,"['Robin Williams', 'Jonathan Hyde', 'Kirsten D...","['board game', 'disappearance', ""based on chil..."


In [48]:
md.shape

(46628, 27)

In [49]:
md = md.rename(columns={"id":"tmdbId"})

In [50]:
md.head(2)

,adult,belongs_to_collection,budget,genres,homepage,tmdbId,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,director,cast,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,John Lasseter,"['Tom Hanks', 'Tim Allen', 'Don Rickles']","['jealousy', 'toy', 'boy', 'friendship', 'frie..."
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,Joe Johnston,"['Robin Williams', 'Jonathan Hyde', 'Kirsten D...","['board game', 'disappearance', ""based on chil..."


In [51]:
md['tmdbId'].dtypes

dtype('int64')

**Using small meta data to train model**

In [52]:
links = pd.read_csv(PATH_TO_DATA + "/links.csv")
links = links[links['tmdbId'].notnull()]['tmdbId'].astype('int')

In [53]:
links.head(5)

0      862
1     8844
2    15602
3    31357
4    11862
Name: tmdbId, dtype: int64

In [54]:
links.shape

(9734,)

In [55]:
smd = md[md['tmdbId'].isin(links)]
smd.shape

(9657, 27)

In [56]:
smd.head(2)

,adult,belongs_to_collection,budget,genres,homepage,tmdbId,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,director,cast,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,John Lasseter,"['Tom Hanks', 'Tim Allen', 'Don Rickles']","['jealousy', 'toy', 'boy', 'friendship', 'frie..."
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,Joe Johnston,"['Robin Williams', 'Jonathan Hyde', 'Kirsten D...","['board game', 'disappearance', ""based on chil..."


In [57]:
# convert string to list

smd['cast'] = smd['cast'].apply(literal_eval)

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [58]:
smd['cast'][0]

['Tom Hanks', 'Tim Allen', 'Don Rickles']

In [59]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [60]:
smd['cast'][0]

['tomhanks', 'timallen', 'donrickles']

In [61]:
# Mention Director 3 times to give it more weight relative to the entire cast.

smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x,x, x])

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [62]:
smd['director'][0]

['johnlasseter', 'johnlasseter', 'johnlasseter']

**Extract Keywords**

In [63]:
# convert sting to list
smd['keywords'] = smd['keywords'].apply(literal_eval)

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [64]:
smd['keywords'].head(2)

0    [jealousy, toy, boy, friendship, friends, riva...
1    [board game, disappearance, based on children'...
Name: keywords, dtype: object

In [65]:
s = smd.apply(lambda x: pd.Series(x['keywords']) ,axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [66]:
s.head()

0      jealousy
0           toy
0           boy
0    friendship
0       friends
Name: keyword, dtype: object

In [67]:
s = s.value_counts()
s[:5]

independent film        613
woman director          573
murder                  398
based on novel          336
duringcreditsstinger    330
Name: keyword, dtype: int64

In [68]:
s = s[s > 1]

In [69]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [70]:
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [71]:
smd['keywords'].head()

0    [jealousy, toy, boy, friendship, friends, riva...
1    [boardgame, disappearance, basedonchildren'sbo...
2          [fishing, bestfriend, duringcreditsstinger]
3    [basedonnovel, interracialrelationship, single...
4    [baby, midlifecrisis, confidence, aging, daugh...
Name: keywords, dtype: object

In [72]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [73]:
smd['soup'].head() # list

0    [jealousy, toy, boy, friendship, friends, riva...
1    [boardgame, disappearance, basedonchildren'sbo...
2    [fishing, bestfriend, duringcreditsstinger, wa...
3    [basedonnovel, interracialrelationship, single...
4    [baby, midlifecrisis, confidence, aging, daugh...
Name: soup, dtype: object

In [74]:
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [75]:
smd['soup'].head() # string

0    jealousy toy boy friendship friends rivalry bo...
1    boardgame disappearance basedonchildren'sbook ...
2    fishing bestfriend duringcreditsstinger walter...
3    basedonnovel interracialrelationship singlemot...
4    baby midlifecrisis confidence aging daughter m...
Name: soup, dtype: object

In [76]:
smd['soup'][0]

'jealousy toy boy friendship friends rivalry boynextdoor newtoy toycomestolife tomhanks timallen donrickles johnlasseter johnlasseter johnlasseter Animation Comedy Family'

**Modeling**

In [77]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

In [78]:
count_matrix.shape

(9657, 112362)

In [79]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [80]:
cosine_sim[0]

array([1.        , 0.02441931, 0.02738955, ..., 0.02934836, 0.03698634,
       0.03049971])

In [81]:
smd.head(2)

,adult,belongs_to_collection,budget,genres,homepage,tmdbId,imdb_id,original_language,original_title,overview,...,status,tagline,title,video,vote_average,vote_count,director,cast,keywords,soup
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Released,NaN,Toy Story,False,7.7,5415.0,"[johnlasseter, johnlasseter, johnlasseter]","[tomhanks, timallen, donrickles]","[jealousy, toy, boy, friendship, friends, riva...",jealousy toy boy friendship friends rivalry bo...
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[joejohnston, joejohnston, joejohnston]","[robinwilliams, jonathanhyde, kirstendunst]","[boardgame, disappearance, basedonchildren'sbo...",boardgame disappearance basedonchildren'sbook ...


In [82]:
smd = smd.reset_index()

In [83]:
smd.head(2)

,index,adult,belongs_to_collection,budget,genres,homepage,tmdbId,imdb_id,original_language,original_title,...,status,tagline,title,video,vote_average,vote_count,director,cast,keywords,soup
0,0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,...,Released,NaN,Toy Story,False,7.7,5415.0,"[johnlasseter, johnlasseter, johnlasseter]","[tomhanks, timallen, donrickles]","[jealousy, toy, boy, friendship, friends, riva...",jealousy toy boy friendship friends rivalry bo...
1,1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,...,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[joejohnston, joejohnston, joejohnston]","[robinwilliams, jonathanhyde, kirstendunst]","[boardgame, disappearance, basedonchildren'sbo...",boardgame disappearance basedonchildren'sbook ...


In [84]:
smd.shape

(9657, 29)

In [85]:
titles = smd['title']
tmdbId = smd['tmdbId']
indices = pd.Series(smd.index, index=smd['tmdbId'])

In [86]:
def get_recommendations(tmdbid):
    try:
        idx = indices[tmdbid] # tmdbId value
        sim_scores = list(enumerate(cosine_sim[idx])) # tmdbId的 value作為 index 取出該 tmdbId的 cosine_sim array
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) # sim_scores[0] = (0, 1.0000000000000018), x[1] 取出 1.0000000000000018
        sim_scores = sim_scores[1:6]
        movie_indices = [i[0] for i in sim_scores]
        movie_scores = [i[1] for i in sim_scores]
        
        tmdb_list = list(tmdbId.iloc[movie_indices])
        title_list = list(titles.iloc[movie_indices])
        score_list = movie_scores
        
        csim_movies = []
        for a,b,c in zip(tmdb_list, title_list, score_list):
            csim_movies.append(str(a) + '|' +  b + '|' + str(round(c,3)))
        
        return csim_movies
    except:
        return []
        pass

In [87]:
get_recommendations(862) # tmdbId|movie title|sim_score

['13925|Luxo Jr.|0.555',
 '863|Toy Story 2|0.512',
 '49013|Cars 2|0.429',
 '920|Cars|0.391',
 "9487|A Bug's Life|0.384"]

**Get Recommendation for all movies**

In [88]:
df_cb.head(2)

,movieId,tmdbId,title,release_date,genres,poster_path,runtime,budget,popularity,revenue,vote_average,vote_count
0,1,862,Toy Story,1995,"[adventure, animation, children, comedy, fantasy]",/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,81,30000000,21,373554033,7.7,5415
1,2,8844,Jumanji,1995,"[adventure, children, fantasy]",/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,104,65000000,17,262797249,6.9,2413


In [89]:
df_final = df_cb.copy()

In [90]:
df_final.head(2)

,movieId,tmdbId,title,release_date,genres,poster_path,runtime,budget,popularity,revenue,vote_average,vote_count
0,1,862,Toy Story,1995,"[adventure, animation, children, comedy, fantasy]",/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,81,30000000,21,373554033,7.7,5415
1,2,8844,Jumanji,1995,"[adventure, children, fantasy]",/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,104,65000000,17,262797249,6.9,2413


In [91]:
df_final['csim_movies'] = df_final['tmdbId'].apply(get_recommendations)

In [92]:
df_final.head(3)

,movieId,tmdbId,title,release_date,genres,poster_path,runtime,budget,popularity,revenue,vote_average,vote_count,csim_movies
0,1,862,Toy Story,1995,"[adventure, animation, children, comedy, fantasy]",/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,81,30000000,21,373554033,7.7,5415,"[13925|Luxo Jr.|0.555, 863|Toy Story 2|0.512, ..."
1,2,8844,Jumanji,1995,"[adventure, children, fantasy]",/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,104,65000000,17,262797249,6.9,2413,"[15139|The Pagemaster|0.433, 9354|Honey, I Shr..."
2,3,15602,Grumpier Old Men,1995,"[comedy, romance]",/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,101,0,11,0,6.5,92,"[27472|The Odd Couple II|0.553, 41579|Getting ..."


## Step 5: Convert Pandas Dataframe to Spark Dataframe

In [94]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)

In [95]:
spark_df = sqlContext.createDataFrame(df_final)

In [96]:
spark_df.show(2)

+-------+------+---------+------------+--------------------+--------------------+-------+--------+----------+---------+-----------------+----------+--------------------+
|movieId|tmdbId|    title|release_date|              genres|         poster_path|runtime|  budget|popularity|  revenue|     vote_average|vote_count|         csim_movies|
+-------+------+---------+------------+--------------------+--------------------+-------+--------+----------+---------+-----------------+----------+--------------------+
|      1|   862|Toy Story|        1995|[adventure, anima...|/rhIRbceoE9lR4veE...|     81|30000000|        21|373554033|7.699999809265137|      5415|[13925|Luxo Jr.|0...|
|      2|  8844|  Jumanji|        1995|[adventure, child...|/vzmL6fP7aPKNKPRT...|    104|65000000|        17|262797249|6.900000095367432|      2413|[15139|The Pagema...|
+-------+------+---------+------------+--------------------+--------------------+-------+--------+----------+---------+-----------------+----------+--

In [97]:
spark_df.dtypes

[('movieId', 'bigint'),
 ('tmdbId', 'bigint'),
 ('title', 'string'),
 ('release_date', 'string'),
 ('genres', 'array<string>'),
 ('poster_path', 'string'),
 ('runtime', 'bigint'),
 ('budget', 'bigint'),
 ('popularity', 'bigint'),
 ('revenue', 'bigint'),
 ('vote_average', 'double'),
 ('vote_count', 'bigint'),
 ('csim_movies', 'array<string>')]

**Convert data types**

In [98]:
spark_df = spark_df.select(
    spark_df.movieId.cast("int"), spark_df.tmdbId.cast("int"), "title",\
    "release_date", "genres", "poster_path", spark_df.runtime.cast("int"),\
    spark_df.budget.cast("int"), spark_df.popularity.cast("int"), spark_df.revenue.cast("int"), \
    spark_df.vote_average.cast("float"), spark_df.vote_count.cast("int"), "csim_movies"
)
print("Cleaned movie data:")
spark_df.show(5)

Cleaned movie data:
+-------+------+--------------------+------------+--------------------+--------------------+-------+--------+----------+---------+------------+----------+--------------------+
|movieId|tmdbId|               title|release_date|              genres|         poster_path|runtime|  budget|popularity|  revenue|vote_average|vote_count|         csim_movies|
+-------+------+--------------------+------------+--------------------+--------------------+-------+--------+----------+---------+------------+----------+--------------------+
|      1|   862|           Toy Story|        1995|[adventure, anima...|/rhIRbceoE9lR4veE...|     81|30000000|        21|373554033|         7.7|      5415|[13925|Luxo Jr.|0...|
|      2|  8844|             Jumanji|        1995|[adventure, child...|/vzmL6fP7aPKNKPRT...|    104|65000000|        17|262797249|         6.9|      2413|[15139|The Pagema...|
|      3| 15602|    Grumpier Old Men|        1995|   [comedy, romance]|/6ksm1sjKMFLbO7UY...|    101|

In [99]:
spark_df.dtypes

[('movieId', 'int'),
 ('tmdbId', 'int'),
 ('title', 'string'),
 ('release_date', 'string'),
 ('genres', 'array<string>'),
 ('poster_path', 'string'),
 ('runtime', 'int'),
 ('budget', 'int'),
 ('popularity', 'int'),
 ('revenue', 'int'),
 ('vote_average', 'float'),
 ('vote_count', 'int'),
 ('csim_movies', 'array<string>')]

In [100]:
spark_df.count()

9753

## Step 6: Load data into Elasticsearch

Now that you have your dataset processed and prepared, you will load it into Elasticsearch.

_Note:_ for the purposes of this demo notebook you have started with an existing example dataset and will load that into Elasticsearch. In practice you may write your event data as well as user and item metadata from your application directly into Elasticsearch.

First test that your Elasticsearch instance is running and you can connect to it using the Python Elasticsearch client.

In [101]:
from elasticsearch import Elasticsearch

# test your ES instance is running

# es = Elasticsearch([
#     {'host': 'localhost'},
#     {'host': 'othernode', 'port': 443, 'url_prefix': 'es', 'use_ssl': True},
# ])

es = Elasticsearch([{'host':'10.10.114.97', 'port':9200}])
es.info(pretty=True)

{'cluster_name': 'elasticsearch',
 'cluster_uuid': 'u6JNB5P1T-CiALv33_p9uQ',
 'name': 'bMG_2k7',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2017-03-23T03:31:50.652Z',
  'build_hash': '3adb13b',
  'build_snapshot': False,
  'lucene_version': '6.4.1',
  'number': '5.3.0'}}

### Create an Elasticsearch index with mappings for users, movies and rating events

In Elasticsearch, an "index" is roughly similar to a "database", while a "document type" is roughly similar to a "table" in that database. The schema for a document type is called an index mapping.

While Elasticsearch supports dynamic mapping, it's advisable to specify the mapping explicitly when creating an index if you know what your data looks like.

For the purposes of your recommendation engine, this is also necessary so that you can specify a custom analyzer for the field that will hold the recommendation "model" (that is, the factor vectors). This will ensure the vector-scoring plugin will work correctly.

> _Note_ This notebook does not go into detail about the underlying scoring mechanism or the relevant Elasticsearch internals. See the talks and slides in the [Journey Links section](https://github.com/MLnick/elasticsearch-spark-recommender-demo/blob/master/README.md#links) for more detail.

__References:__
* [Create index request](https://www.elastic.co/guide/en/elasticsearch/reference/current/indices-create-index.html)
* [Delimited payload filter](https://www.elastic.co/guide/en/elasticsearch/reference/2.4/analysis-delimited-payload-tokenfilter.html)
* [Term vectors](https://www.elastic.co/guide/en/elasticsearch/reference/2.4/docs-termvectors.html#_term_information)
* [Mapping](https://www.elastic.co/guide/en/elasticsearch/reference/2.4/mapping.html)

> **_Optional_**

> If you are re-running the notebook and have previously created the `demo` index in Elasticsearch, you should first delete it by un-commenting and running the next cell, before running the index creation cell that follows.

In [116]:
# Delete Elasticsearch index if exist

es.indices.delete(index="demo")

{'acknowledged': True}

> **Create Elasticsearch index**

In [117]:
create_index = {
    "settings": {
        "analysis": {
            "analyzer": {
                # this configures the custom analyzer we need to parse vectors such that the scoring
                # plugin will work correctly
                "payload_analyzer": {
                    "type": "custom",
                    "tokenizer":"whitespace",
                    "filter":"delimited_payload_filter"
                }
            }
        }
    },
    "mappings": {
        "ratings": {
          # this mapping definition sets up the fields for the rating events
          "properties": {
                "timestamp": {
                    "type": "date"
                },
                "userId": {
                    "type": "integer"
                },
                "movieId": {
                    "type": "integer"
                },
                "rating": {
                    "type": "double"
                }
            }  
        },
        "users": {
            # this mapping definition sets up the metadata fields for the users
            "properties": {
                "userId": {
                    "type": "integer"
                },
                "@model": {
                    # this mapping definition sets up the fields for user factor vectors of our model
                    "properties": {
                        "factor": {
                            "type": "text",
                            "term_vector": "with_positions_offsets_payloads",
                            "analyzer" : "payload_analyzer"
                        },
                        "version": {
                            "type": "keyword"
                        },
                        "timestamp": {
                            "type": "date"
                        }
                    }
                }
            }
        },
        "movies": {
            # this mapping definition sets up the metadata fields for the movies
            "properties": {
                "movieId": {
                    "type": "integer"
                },
                "genres": {
                    "type": "keyword"
                },
                "poster_path": {
                    "type": "text"
                },
                "runtime": {
                    "type": "integer"
                },
                "budget": {
                    "type": "integer"
                },
                "popularity": {
                    "type": "integer"
                },
                "revenue": {
                    "type": "integer"
                },
                "vote_average": {
                    "type": "float"
                },
                "vote_count": {
                    "type": "integer"
                },
                "release_date": {
                    "type": "date",
                    "format": "year"
                },
                "tmdbId": {
                    "type": "keyword"
                },
                "csim_movies": {
                    "type": "text"
                },
                "@model": {
                    # this mapping definition sets up the fields for movie factor vectors of our model
                    "properties": {
                        "factor": {
                            "type": "text",
                            "term_vector": "with_positions_offsets_payloads",
                            "analyzer" : "payload_analyzer"
                        },
                        "version": {
                            "type": "keyword"
                        },
                        "timestamp": {
                            "type": "date"
                        }
                    }
                }
            }
        }
    }
}
# create index with the settings and mappings above
es.indices.create(index="demo", body=create_index)

{'acknowledged': True, 'shards_acknowledged': True}

### Load Ratings and Movies DataFrames into Elasticsearch

First you will write the ratings data to Elasticsearch. Notice that you can simply use the Spark Elasticsearch connector to write a `DataFrame` with the native Spark datasource API by specifying `format("es")`

In [118]:
# write ratings data
ratings.write.format("es").save("demo/ratings")

# check write went ok
print("Dataframe count: %d" % ratings.count())
print("ES index count:  %d" % es.count(index="demo", doc_type="ratings")['count'])

Dataframe count: 100836
ES index count:  100836


In [119]:
# test things out by retrieving a few rating event documents from Elasticsearch
es.search(index="demo", doc_type="ratings", q="*", size=3)

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'AWe68gO7HdazyX_fbWPt',
    '_index': 'demo',
    '_score': 1.0,
    '_source': {'movieId': 1,
     'rating': 4.0,
     'timestamp': 964982703000,
     'userId': 1},
    '_type': 'ratings'},
   {'_id': 'AWe68gO7HdazyX_fbWPw',
    '_index': 'demo',
    '_score': 1.0,
    '_source': {'movieId': 47,
     'rating': 5.0,
     'timestamp': 964983815000,
     'userId': 1},
    '_type': 'ratings'},
   {'_id': 'AWe68gO7HdazyX_fbWP5',
    '_index': 'demo',
    '_score': 1.0,
    '_source': {'movieId': 223,
     'rating': 3.0,
     'timestamp': 964980985000,
     'userId': 1},
    '_type': 'ratings'}],
  'max_score': 1.0,
  'total': 100836},
 'timed_out': False,
 'took': 4}

**Write the movie metadata**

In [120]:
# write movie data, specifying the DataFrame column to use as the id mapping
spark_df.write.format("es").option("es.mapping.id", "movieId").save("demo/movies")

# check load went ok
print("Movie DF count: %d" % spark_df.count())
print("ES index count: %d" % es.count(index="demo", doc_type="movies")['count'])

Movie DF count: 9753
ES index count: 9742


Again you can harness the power of search to query the movie metadata:

In [121]:
# test things out by searching for movies containing "matrix" in the title
# es.search(index="demo", doc_type="movies", q="title:matrix", size=3)
es.search(index="demo", doc_type="movies", q="title:Catwalk", size=3)

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '108',
    '_index': 'demo',
    '_score': 10.017448,
    '_source': {'budget': 0,
     'csim_movies': ['295538|The Culture High|0.091',
      '22010|Grass|0.084',
      '18603|Hands on a Hard Body: The Documentary|0.081',
      '35199|Wordplay|0.081',
      '14358|Mad Hot Ballroom|0.078'],
     'genres': ['documentary'],
     'movieId': 108,
     'popularity': 0,
     'poster_path': '/yhFcbTCnsWjg3nH3PLL6RoltjqS.jpg',
     'release_date': '1996',
     'revenue': 0,
     'runtime': 95,
     'title': 'Catwalk',
     'tmdbId': 89333,
     'vote_average': 7.0,
     'vote_count': 2},
    '_type': 'movies'}],
  'max_score': 10.017448,
  'total': 1},
 'timed_out': False,
 'took': 2}

## Step 7: Train a recommmender model on the ratings data

Your data is now stored in Elasticsearch and you will use the ratings data to build a collaborative filtering recommendation model.

[Collaborative filtering](https://en.wikipedia.org/wiki/Collaborative_filtering) is a recommendation approach that is effectively based on the "wisdom of the crowd". It makes the assumption that, if two people share similar preferences, then the things that one of them prefers could be good recommendations to make to the other. In other words, if user A tends to like certain movies, and user B shares some of these preferences with user A, then the movies that user A likes, that user B _has not yet seen_, may well be movies that user B will also like.

In a similar manner, we can think about _items_ as being similar if they tend to be rated highly by the same people, on average. 

Hence these models are based on the combined, collaborative preferences and behavior of all users in aggregate. They tend to be very effective in practice (provided you have enough preference data to train the model). The ratings data you have is a form of _explicit preference data_, perfect for training collaborative filtering models.

### Alternating Least Squares

Alternating Least Squares (ALS) is a specific algorithm for solving a type of collaborative filtering model known as [matrix factorization (MF)](https://en.wikipedia.org/wiki/Matrix_decomposition). The core idea of MF is to represent the ratings as a _user-item ratings matrix_. In the diagram below you will see this matrix on the left (with users as _rows_ and movies as _columns_). The entries in this matrix are the ratings given by users to movies.

You may also notice that the matrix has _missing entries_ because not all users have rated all movies. In this situation we refer to the data as _sparse_.

![als-diagram.png](../doc/source/images/als-diagram.png)

MF methods aim to find two much smaller matrices (one representing the _users_ and the other the _items_) that, when multiplied together, re-construct the original ratings matrix as closely as possible. This is know as _factorizing_ the original matrix, hence the name of the technique.

The two smaller matrices are called _factor matrices_ (or _latent features_). The user and movie factor matrices are illustrated on the right in the diagram above. The idea is that each user factor vector is a compressed representation of the user's preferences and behavior. Likewise, each item factor vector is a compressed representation of the item. Once the model is trained, the factor vectors can be used to make recommendations, which is what you will do in the following sections.

__Further reading:__

* [Spark MLlib Collaborative Filtering](http://spark.apache.org/docs/latest/ml-collaborative-filtering.html)
* [Alternating Least Squares and collaborative filtering](https://datasciencemadesimpler.wordpress.com/tag/alternating-least-squares/)
* [Quora question on Alternating Least Squares](https://www.quora.com/What-is-the-Alternating-Least-Squares-method-in-recommendation-systems-And-why-does-this-algorithm-work-intuition-behind-this)

Fortunately, Spark's MLlib machine learning library has a scalable, efficient implementation of matrix factorization built in, which we can use to train our recommendation model. Next, you will use Spark's ALS to train a model on your ratings data from Elasticsearch.

In [122]:
ratings_from_es = spark.read.format("es").load("demo/ratings")
ratings_from_es.show(5)

+-------+------+-------------------+------+
|movieId|rating|          timestamp|userId|
+-------+------+-------------------+------+
|      6|   4.0|2000-07-30 14:37:04|     1|
|     50|   5.0|2000-07-30 14:48:51|     1|
|    231|   5.0|2000-07-30 14:19:39|     1|
|    296|   3.0|2000-07-30 14:49:27|     1|
|    349|   4.0|2000-07-30 14:42:43|     1|
+-------+------+-------------------+------+
only showing top 5 rows



**Modeling**

In [123]:
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import col
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", regParam=0.01, rank=20, seed=12)
model = als.fit(ratings_from_es)
model.userFactors.show(5)
model.itemFactors.show(5)

+---+--------------------+
| id|            features|
+---+--------------------+
| 10|[-1.2723869, -1.0...|
| 20|[-0.28433216, -0....|
| 30|[-0.23606953, 0.6...|
| 40|[0.31492335, -0.0...|
| 50|[0.006268716, 0.0...|
+---+--------------------+
only showing top 5 rows

+---+--------------------+
| id|            features|
+---+--------------------+
| 10|[0.07222334, 0.37...|
| 20|[-0.40369913, 0.5...|
| 30|[-0.65819603, -0....|
| 40|[-0.2619177, 0.49...|
| 50|[-0.46155798, 0.1...|
+---+--------------------+
only showing top 5 rows



## Step 8: Export ALS user and item factor vectors to Elasticsearch

Congratulations, you've trained a recommendation model! The next step is to export the model factors (shown in the `DataFrames` above) to Elasticsearch.

In order to store the model in the correct format for the index mappings set up earlier, you will need to create some utility functions. These functions will allow you to convert the raw vectors (which are equivalent to a Python list in the factor `DataFrames` above) to the correct _delimited string format_. This ensures Elasticsearch will parse the vector field in the model correctly using the delimited token filter custom analyzer you configured earlier.

You will also create a function to convert a vector and related metadata (such as the Spark model id and a timestamp) into a `DataFrame` field that matches the `model` field in the Elasticsearch index mapping.

### Utility functions for converting factor vectors

In [124]:
from pyspark.sql.types import *
from pyspark.sql.functions import udf, lit, current_timestamp, unix_timestamp

def convert_vector(x):
    '''Convert a list or numpy array to delimited token filter format'''
    return " ".join(["%s|%s" % (i, v) for i, v in enumerate(x)])

def reverse_convert(s):
    '''Convert a delimited token filter format string back to list format'''
    return  [float(f.split("|")[1]) for f in s.split(" ")]

def vector_to_struct(x, version, ts):
    '''Convert a vector to a SparkSQL Struct with string-format vector and version fields'''
    return (convert_vector(x), version, ts)

vector_struct = udf(vector_to_struct, \
                    StructType([StructField("factor", StringType(), True), \
                                StructField("version", StringType(), True),\
                                StructField("timestamp", LongType(), True)]))

In [125]:
# test out the vector conversion function
test_vec = model.userFactors.select("features").first().features
print(test_vec)
print()
print(convert_vector(test_vec))

[-1.272386908531189, -1.040471076965332, 0.6506963968276978, -0.7358267307281494, 0.2215612679719925, -0.24009384214878082, -0.03175492584705353, -0.1631995439529419, 0.3178538382053375, -0.18343225121498108, -0.6073395609855652, 0.30508336424827576, 0.44866761565208435, -0.2556912899017334, 0.1966400444507599, -0.0496751107275486, -0.47266942262649536, -0.45564523339271545, -0.3641686737537384, -0.40033015608787537]

0|-1.272386908531189 1|-1.040471076965332 2|0.6506963968276978 3|-0.7358267307281494 4|0.2215612679719925 5|-0.24009384214878082 6|-0.03175492584705353 7|-0.1631995439529419 8|0.3178538382053375 9|-0.18343225121498108 10|-0.6073395609855652 11|0.30508336424827576 12|0.44866761565208435 13|-0.2556912899017334 14|0.1966400444507599 15|-0.0496751107275486 16|-0.47266942262649536 17|-0.45564523339271545 18|-0.3641686737537384 19|-0.40033015608787537


### Convert factor vectors to [factor, version, timestamp] form and write to Elasticsearch

In [126]:
ver = model.uid
ts = unix_timestamp(current_timestamp())
movie_vectors = model.itemFactors.select("id", vector_struct("features", lit(ver), ts).alias("@model"))
movie_vectors.select("id", "@model.factor", "@model.version", "@model.timestamp").show(5)

user_vectors = model.userFactors.select("id", vector_struct("features", lit(ver), ts).alias("@model"))
user_vectors.select("id", "@model.factor", "@model.version", "@model.timestamp").show(5)

+---+--------------------+--------------------+----------+
| id|              factor|             version| timestamp|
+---+--------------------+--------------------+----------+
| 10|0|0.0722233429551...|ALS_4bfe8413b88ca...|1545029714|
| 20|0|-0.403699129819...|ALS_4bfe8413b88ca...|1545029714|
| 30|0|-0.658196032047...|ALS_4bfe8413b88ca...|1545029714|
| 40|0|-0.261917710304...|ALS_4bfe8413b88ca...|1545029714|
| 50|0|-0.461557984352...|ALS_4bfe8413b88ca...|1545029714|
+---+--------------------+--------------------+----------+
only showing top 5 rows

+---+--------------------+--------------------+----------+
| id|              factor|             version| timestamp|
+---+--------------------+--------------------+----------+
| 10|0|-1.272386908531...|ALS_4bfe8413b88ca...|1545029715|
| 20|0|-0.284332156181...|ALS_4bfe8413b88ca...|1545029715|
| 30|0|-0.236069530248...|ALS_4bfe8413b88ca...|1545029715|
| 40|0|0.3149233460426...|ALS_4bfe8413b88ca...|1545029715|
| 50|0|0.0062687159515...|ALS_4

## Step 9 : Save the model to Elasticsearch

In [127]:
# write data to ES, use:
# - "id" as the column to map to ES movie id
# - "update" write mode for ES, since you want to update new fields only
# - "append" write mode for Spark
movie_vectors.write.format("es") \
    .option("es.mapping.id", "id") \
    .option("es.write.operation", "update") \
    .save("demo/movies", mode="append")

In [128]:
# write data to ES, use:
# - "id" as the column to map to ES movie id
# - "index" write mode for ES, since you have not written to the user index previously
# - "append" write mode for Spark
user_vectors.write.format("es") \
    .option("es.mapping.id", "id") \
    .option("es.write.operation", "index") \
    .save("demo/users", mode="append")

### Check the data  was written correctly

You can search for a movie to see if the model factor vector was written correctly. You should see a `'@model': {'factor': '0|...` field in the returned movie document.

In [129]:
# search for a particular sci-fi movie
es.search(index="demo", doc_type="movies", q="star wars phantom menace", size=1)['hits']['hits'][0]

{'_id': '2628',
 '_index': 'demo',
 '_score': 12.851084,
 '_source': {'@model': {'factor': '0|0.5367742776870728 1|1.315629482269287 2|0.908174991607666 3|-0.17290841042995453 4|0.6622549891471863 5|-0.1442941427230835 6|-0.43587976694107056 7|-0.5659969449043274 8|0.7238355278968811 9|-0.9211186170578003 10|1.0284417867660522 11|0.6716753840446472 12|0.9536047577857971 13|0.07491487264633179 14|-1.738380789756775 15|0.9156777262687683 16|-1.43646240234375 17|0.1269250065088272 18|-0.7548630833625793 19|-2.6259231567382812',
   'timestamp': 1545029717,
   'version': 'ALS_4bfe8413b88ca97af096'},
  'budget': 115000000,
  'csim_movies': ['1894|Star Wars: Episode II - Attack of the Clones|0.555',
   '1895|Star Wars: Episode III - Revenge of the Sith|0.512',
   '11|Star Wars|0.465',
   '636|THX 1138|0.356',
   '838|American Graffiti|0.283'],
  'genres': ['action', 'adventure', 'sci-fi'],
  'id': 2628,
  'movieId': 2628,
  'popularity': 15,
  'poster_path': '/n8V09dDc02KsSN6Q4hC2BX6hN8X.jpg'

In [130]:
es.search(index="demo", doc_type="movies", q="title:matrix", size=1)['hits']['hits'][0]

{'_id': '2571',
 '_index': 'demo',
 '_score': 6.9453225,
 '_source': {'@model': {'factor': '0|-0.07852364331483841 1|0.7339636087417603 2|1.4943597316741943 3|0.5051127672195435 4|1.6120359897613525 5|-0.10694307088851929 6|-0.33216172456741333 7|-1.9453973770141602 8|0.6144668459892273 9|-1.1312552690505981 10|1.208752155303955 11|-0.5731072425842285 12|0.6501845121383667 13|0.257805734872818 14|-1.8023306131362915 15|0.4234260022640228 16|-1.9008692502975464 17|0.25924360752105713 18|-0.5445372462272644 19|-3.1181275844573975',
   'timestamp': 1545029717,
   'version': 'ALS_4bfe8413b88ca97af096'},
  'budget': 63000000,
  'csim_movies': ['604|The Matrix Reloaded|0.362',
   '605|The Matrix Revolutions|0.353',
   '9303|Bound|0.272',
   '8870|Red Planet|0.183',
   '55931|The Animatrix|0.181'],
  'genres': ['action', 'sci-fi', 'thriller'],
  'id': 2571,
  'movieId': 2571,
  'popularity': 33,
  'poster_path': '/hEpWvX6Bp79eLxY1kX5ZZJcme5U.jpg',
  'release_date': '1999',
  'revenue': 463517

In [132]:
help(es)

Help on Elasticsearch in module elasticsearch.client object:

class Elasticsearch(builtins.object)
 |  Elasticsearch low-level client. Provides a straightforward mapping from
 |  Python to ES REST endpoints.
 |  
 |  The instance has attributes ``cat``, ``cluster``, ``indices``, ``ingest``,
 |  ``nodes``, ``snapshot`` and ``tasks`` that provide access to instances of
 |  :class:`~elasticsearch.client.CatClient`,
 |  :class:`~elasticsearch.client.ClusterClient`,
 |  :class:`~elasticsearch.client.IndicesClient`,
 |  :class:`~elasticsearch.client.IngestClient`,
 |  :class:`~elasticsearch.client.NodesClient`,
 |  :class:`~elasticsearch.client.SnapshotClient` and
 |  :class:`~elasticsearch.client.TasksClient` respectively. This is the
 |  preferred (and only supported) way to get access to those classes and their
 |  methods.
 |  
 |  You can specify your own connection class which should be used by providing
 |  the ``connection_class`` parameter::
 |  
 |      # create connection to local